<a href="https://colab.research.google.com/github/Ghanshyam-02/btech-project-pinn/blob/main/pinn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyDOE

In [ ]:
# Physics-Informed Neural Network for the swing equation
# Converted to TensorFlow 2.x from the original TensorFlow 1.x implementation
# Author: assistant (converted for user)
# Usage: place the same ../Data/swingEquation_inference.mat next to this script and run with python3

import tensorflow as tf
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import time
from pyDOE import lhs

# reproducibility
np.random.seed(1234)
tf.random.set_seed(1234)

class PINN:
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        self.lb = tf.constant(lb.reshape((1, -1)), dtype=tf.float32)
        self.ub = tf.constant(ub.reshape((1, -1)), dtype=tf.float32)

        self.x_u = tf.convert_to_tensor(X_u[:,0:1], dtype=tf.float32)
        self.t_u = tf.convert_to_tensor(X_u[:,1:2], dtype=tf.float32)
        self.u = tf.convert_to_tensor(u, dtype=tf.float32)

        self.x_f = tf.convert_to_tensor(X_f[:,0:1], dtype=tf.float32)
        self.t_f = tf.convert_to_tensor(X_f[:,1:2], dtype=tf.float32)

        self.layers = layers
        self.nu = nu

        # build neural network (Keras sequential with tanh activations)
        self.model = self.build_model(layers)

        # optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

    def build_model(self, layers):
        model = tf.keras.Sequential()
        # input scaling layer (to [-1,1]) handled inside call
        for i in range(len(layers)-1):
            units = layers[i+1]
            if i < len(layers)-2:
                model.add(tf.keras.layers.Dense(units, activation='tanh',
                                                kernel_initializer=tf.keras.initializers.GlorotNormal()))
            else:
                # last layer linear
                model.add(tf.keras.layers.Dense(units, activation=None,
                                                kernel_initializer=tf.keras.initializers.GlorotNormal()))
        return model

    def neural_net(self, X):
        # X shape: (N,2) where columns are x and t
        # scale X to [-1,1]
        X_scaled = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        return self.model(X_scaled)

    def net_u(self, x, t):
        X = tf.concat([x, t], axis=1)
        return self.neural_net(X)

    def net_f(self, x, t):
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch([x, t])
            with tf.GradientTape(persistent=True) as tape:
                tape.watch([x, t])
                u = self.net_u(x, t)
            u_t = tape.gradient(u, t)
        u_tt = tape2.gradient(u_t, t)
        # free memory
        del tape
        del tape2

        # f = 0.4*u_tt + 0.15*u_t + nu*sin(u) - x
        f = 0.4*u_tt + 0.15*u_t + self.nu*tf.math.sin(u) - x
        return f

    @tf.function
    def loss_fn(self):
        # data loss
        u_pred = self.net_u(self.x_u, self.t_u)
        mse_u = tf.reduce_mean(tf.square(self.u - u_pred))
        # physics loss
        f_pred = self.net_f(self.x_f, self.t_f)
        mse_f = tf.reduce_mean(tf.square(f_pred))
        return mse_u + mse_f, mse_u, mse_f

    @tf.function
    def train_step(self):
        with tf.GradientTape() as tape:
            loss, mse_u, mse_f = self.loss_fn()
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss, mse_u, mse_f

    def train(self, epochs=20000, print_every=1000):
        start = time.time()
        for epoch in range(1, epochs+1):
            loss, mse_u, mse_f = self.train_step()
            if epoch % print_every == 0 or epoch == 1:
                print(f"Epoch {epoch:6d} | Loss: {loss.numpy():.6e} | mse_u: {mse_u.numpy():.6e} | mse_f: {mse_f.numpy():.6e}")
        print('Training time: {:.4f} seconds'.format(time.time()-start))

    def predict(self, X_star):
        x_star = tf.convert_to_tensor(X_star[:,0:1], dtype=tf.float32)
        t_star = tf.convert_to_tensor(X_star[:,1:2], dtype=tf.float32)
        u_star = self.net_u(x_star, t_star).numpy()
        f_star = self.net_f(x_star, t_star).numpy()
        return u_star, f_star


if __name__ == "__main__":
    nu = 0.2

    # load data (same file path as original code)
    data = scipy.io.loadmat("/swingEquation_inference.mat")
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = np.real(data['usol']).T

    X, T = np.meshgrid(x, t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.flatten()[:,None]

    # Domain bounds
    lb = np.array([0.08, 0.0])
    ub = np.array([0.18, 20.0])

    # boundary / initial data (same strategy as original)
    xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))   # t across x at t=0 ??? (note: kept same slicing as original)
    uu1 = Exact[0:1,:].T
    xx2 = np.hstack((X[:,0:1], T[:,0:1]))
    uu2 = Exact[:,0:1]
    xx3 = np.hstack((X[:,-1:], T[:,-1:]))
    uu3 = Exact[:,-1:]

    X_u_train = np.vstack([xx1, xx2, xx3])
    N_f = 8000
    X_f_train = lb + (ub-lb)*lhs(2, N_f)
    X_f_train = np.vstack((X_f_train, X_u_train))

    u_train = np.vstack([uu1, uu2, uu3])

    # pick N_u points
    N_u = 40
    idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
    X_u_train = X_u_train[idx, :]
    u_train = u_train[idx,:]

    layers = [2, 10, 10, 10, 10, 10, 1]

    model = PINN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)

    # train
    model.train(epochs=8000, print_every=500)

    # predict on whole grid
    u_pred, f_pred = model.predict(X_star)

    # compute error
    error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    print('Relative L2 error u: %e' % (error_u))

    # reshape for plotting
    U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')

    # choose a p1 value (x position) to compare time series
    # change p1 to any x value from the data's x array (e.g. 0.1)
    p1 = 0.10
    # find nearest x column index
    ix = np.argmin(np.abs(x.flatten() - p1))
    print(f"Selected p1 = {x.flatten()[ix]:.6f} (closest available x)")

    # plot comparison: u_exact vs u_pred at x = p1 across time
    plt.figure(figsize=(8,4))
    plt.plot(t.flatten(), Exact[:, ix], label='Exact usol')
    plt.plot(t.flatten(), U_pred[:, ix], '--', label='PINN u_pred')
    plt.xlabel('Time')
    plt.ylabel('u')
    plt.title(f'Comparison at x = {x.flatten()[ix]:.6f}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 2D plot (surface) of u_pred over (x,t)
    plt.figure(figsize=(8,5))
    cs = plt.pcolormesh(X, T, U_pred, shading='auto')
    plt.xlabel('x')
    plt.ylabel('t')
    plt.title('PINN prediction U_pred(x,t)')
    plt.colorbar(cs, label='u')
    plt.tight_layout()
    plt.show()

    # Save results if needed
    # np.savez('pinn_results.npz', X=X, T=T, U_pred=U_pred, Exact=Exact)

    print('Done.')


FileNotFoundError: [Errno 2] No such file or directory: '/swingEquation_inference.mat'

[link text](https://)# New Section

In [ ]:
# -----------------------------------------------------------
# Plot u_pred vs time for ALL x values
# -----------------------------------------------------------

plt.figure(figsize=(10,6))
for i in range(X.shape[1]):
    plt.plot(t.flatten(), U_pred[:, i], alpha=0.5)  # PINN predictions
plt.xlabel('Time')
plt.ylabel('u_pred')
plt.title('PINN predictions for all x values (100 curves)')
plt.grid(True)
plt.tight_layout()
plt.show()

# -----------------------------------------------------------
# Plot both u_pred and Exact vs time for ALL x values
# (overlay curves to compare visually)
# -----------------------------------------------------------

plt.figure(figsize=(10,6))
for i in range(X.shape[1]):
    plt.plot(t.flatten(), Exact[:, i], 'k', alpha=0.3)        # exact solution (black)
    plt.plot(t.flatten(), U_pred[:, i], 'r--', alpha=0.3)     # predicted (red dashed)
plt.xlabel('Time')
plt.ylabel('u')
plt.title('Comparison of PINN vs Exact for all x values')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
# Select 5 random p1 points
# -----------------------------------------------------------
np.random.seed(1234)  # for reproducibility
p1_indices = np.random.choice(X.shape[1], size=5, replace=False)
print("Selected p1 indices:", p1_indices)

# -----------------------------------------------------------
# Compute frequency (rad/s) from predicted and exact rotor angle
# -----------------------------------------------------------
dt = t[1,0] - t[0,0]
freq_pred = np.gradient(U_pred, dt, axis=0)
freq_exact = np.gradient(Exact, dt, axis=0)

# -----------------------------------------------------------
# Plot rotor angle and frequency for 5 points in one figure
# -----------------------------------------------------------
fig, axs = plt.subplots(2, 1, figsize=(12,8), sharex=True)

colors = ['b', 'g', 'r', 'c', 'm']  # for 5 curves

# 1. Rotor angle δ
for idx, i in enumerate(p1_indices):
    axs[0].plot(t.flatten(), Exact[:, i], color=colors[idx], linestyle='-', label=f'Exact p1={x.flatten()[i]:.3f}')
    axs[0].plot(t.flatten(), U_pred[:, i], color=colors[idx], linestyle='--', label=f'PINN p1={x.flatten()[i]:.3f}')
axs[0].set_ylabel('Rotor angle δ [rad]')
axs[0].set_title('Rotor Angle δ for 5 Random p1 Points')
axs[0].grid(True)
axs[0].legend()

# 2. Frequency dδ/dt
for idx, i in enumerate(p1_indices):
    axs[1].plot(t.flatten(), freq_exact[:, i], color=colors[idx], linestyle='-', label=f'Exact p1={x.flatten()[i]:.3f}')
    axs[1].plot(t.flatten(), freq_pred[:, i], color=colors[idx], linestyle='--', label=f'PINN p1={x.flatten()[i]:.3f}')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Frequency [rad/s]')
axs[1].set_title('Frequency Response for 5 Random p1 Points')
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()
